In [1]:
import pandas as pd
df = pd.read_csv('/RecipeNLG_dataset.csv', index_col=[0])

In [2]:
import requests
import json
import re
from bs4 import BeautifulSoup

In [3]:
df.columns

Index(['title', 'ingredients', 'directions', 'link', 'source', 'NER'], dtype='object')

In [4]:
def getPrice(ing):
    ingList = ing.strip('][').replace('\"', '').replace(", ", "\\n")

    widget = requests.post(
        "https://api.spoonacular.com/recipes/visualizePriceEstimator",
        data={
            "apiKey": "17a880d8a40f4451bccdc80f4d3aa38e",
            "ingredientList": ingList,
            "servings": "1",
            "defaultCss": "false",
        },
    )
    soup = BeautifulSoup(widget.text, "html.parser")
    things = soup.select_one("#spoonacularPriceBreakdownTable").find_all("div")

    items = repr(things[2]).split("<br/>")[1:-1]
    prices = repr(things[3]).split("<br/>")[1:-1]

    ingredients = dict(zip(items, prices))
    return ingredients


In [45]:
def totalPrice(prices):
    # Converts Dollars to Float Values
    pricesList = [float(price[1:]) for price in list(prices.values())]
    # Adds individual prices
    totalPrice = sum(pricesList) 
    
    return round(totalPrice, 2)

In [51]:
amount = 50

ingredientPrices = df.ingredients.iloc[:amount].apply(getPrice) 
price = ingredientPrices.apply(totalPrice)

ingredientPrices.name = "Ing. Prices"
price.name = "Price"

In [52]:
outputDf = df.iloc[:amount].join(ingredientPrices).join(price)

In [56]:
pd.DataFrame(ingredientPrices.values[0].keys(), ingredientPrices.values[0].values)

In [55]:
outputDf.head()

,title,ingredients,directions,link,source,NER,Ing. Prices,Price
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu...","{'1 cup firmly brown sugar': '$0.71', 'Â½ cups...",4.28
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom...","{'some cut up': '$0.36', '4 boned chicken brea...",9.69
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar...","{'32 ounces frozen corn': '$4.21', '8 ounces c...",7.27
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo...","{'1 large whole chicken': '$5.06', '21 ounces ...",16.96
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu...","{'1 cup peanut butter': '$0.92', 'Â¾ cups grah...",4.92
